<a href="https://colab.research.google.com/github/athiniotim/Final_Essay_Forest_Fires/blob/main/forest_fires_tovima_articles_scraper_Michaela_Athinioti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import time
import random
from tqdm import tqdm

In [ ]:
def get_webpage_soup(url):
    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    response = requests.get(url, headers=headers)


    if response.status_code != 200:

      return None
    else:
    # Μετατροπή του πηγαίου κώδικα της ιστοσελίδας σε αναζητήσιμο κείμενο (soup)
    # print(f"{url} was scraped successfully")
      return BeautifulSoup(response.text, 'html.parser')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
fire_articles_tovima_df = pd.read_csv('/content/drive/MyDrive/Final Essay ΕΔΔΕ.2 Αθηνιώτη Μιχαέλα/clean csv/forest_fires_tovima_teasers_scraper_csv.csv')
fire_articles_tovima_df.head()

,Unnamed: 0,title,article_url,date,time,author,description
0,0,Φωτιά στη Φθιώτιδα - Επιχειρεί και ελικόπτερο,https://www.tovima.gr/2025/04/16/society/agrot...,16.04.2025,14:51,NaN,Κοντά σε πευκόφυτη περιοχή στον Λόγγο του Αγίο...
1,1,Κεφαλλονιά: Φωτιά σε δασική έκταση στη περιοχή...,https://www.tovima.gr/2025/04/27/society/kefal...,27.04.2025,15:02,NaN,Για την κατάσβεση της φωτιάς στην Κεφαλλονιά ε...
2,2,Φωτιά σε αγροτοδασική έκταση στα Χανιά - Μήνυμ...,https://www.tovima.gr/2025/03/13/society/fotia...,13.03.2025,08:23,NaN,Για την κατάσβεση της φωτιά στα Χανιά επιχειρο...
3,3,Ξάνθη: Φωτιά σε δασική έκταση - Επιχειρούν ενά...,https://www.tovima.gr/2024/09/28/society/ksant...,28.09.2024,16:08,NaN,Μάχη με τις φλόγες της φωτιάς σε δασική έκταση...
4,4,Φωτιά τώρα στη Φθιώτιδα - Καίγεται δασική έκταση,https://www.tovima.gr/2024/09/01/society/fotia...,01.09.2024,18:12,NaN,Φωτιά τώρα ξέσπασε στη Φθιώτιδα - Καίγεται δασ...


In [ ]:
def get_article_elements_from_soup(soup, article_url):
  article_block = soup.find('main', {"class" : "is-relative post-wrapper maxgridrow art-art01"})
  # article url
  article_url = article_url # το παίρνω από το arguement του function
  # title
  try:
    title = article_block.find('h1').text.strip()
  except:
    title=""
  # datetime
  try:
    date_str = article_block.find("time") ["datetime"]
    time_str = article_block.find("time", {'class' : 'line-height-1 post-date manrope lightgrey-c'}).text.strip()
  except:
    date_str = ""
    time_str = ""
  # section
  try:
    section = article_block.find('a', 'parent-category').text.strip()
  except:
    section = ""
  # author
  try:
    author = article_block.find("span", {"class": "vima-author"}).text.strip()
  except:
    author = ""
  # excerpt
  try:
    excert = article_block.find('h2').text.strip()
  except:
    excert=""
  # text body
  try:
    text_body_p_tagsL = article_block.find("div", {"class": "post-body main-content pos-rel article-wrapper"}).find_all("p")
    text_body_textsL = []
    for p_tag in text_body_p_tagsL:
      p_tag_text = p_tag.text.strip()
      text_body_textsL.append(p_tag_text)
      text_body = " ".join(text_body_textsL)
      text_body
  except:
    text_body=""
  # feature image
  try:
    feat_img_link = article_block.find("figure", {"class": "article-image"}).img["src"]
  except:
    feat_img_link=""
  # feature image caption
  try:
    image_caption = article_block.find('h2').text.strip()
  except:
    image_caption=""

  articleD = {"article_url": article_url, "title": title, "date": date_str, "time": time_str, "section": section, "author": author,
              "excert": excert, "text_body": text_body, "feat_img_link": feat_img_link, "feat_img_caption": image_caption}

  return articleD

In [ ]:
def scrape_to_vima_articles_pages(teasers_df):
  all_articles_dataL = []

  for idx, row in tqdm(teasers_df.iterrows(), total=teasers_df.shape[0], desc="Processing rows"):

    article_url = row['article_url']

    soup = get_webpage_soup(article_url)

    articleD = get_article_elements_from_soup(soup, article_url)

    all_articles_dataL.append(articleD)

    delay = random.uniform(1, 3)
    time.sleep(delay)

  df = pd.DataFrame(all_articles_dataL)
  print(f"\nScraping completed. Fetshed {len(df)} articles.")
  return df

In [ ]:
fire_articles_tovima_df = scrape_to_vima_articles_pages(fire_articles_tovima_df)
fire_articles_tovima_df

Processing rows: 100%|██████████| 100/100 [05:40<00:00,  3.41s/it]


Scraping completed. Fetshed 100 articles.


,article_url,title,date,time,section,author,excert,text_body,feat_img_link,feat_img_caption
0,https://www.tovima.gr/2025/04/16/society/agrot...,Αγροτοδασική φωτιά στη Φθιώτιδα – Μεγάλη κινητ...,16-04-2025,14:51,Κοινωνία,Newsroom,Κοντά σε πευκόφυτη περιοχή στον Λόγγο του Αγίο...,Σε συναγερμό έχουν τεθεί οι πυροσβεστικές δυνά...,https://www.tovima.gr/wp-content/uploads/2024/...,Κοντά σε πευκόφυτη περιοχή στον Λόγγο του Αγίο...
1,https://www.tovima.gr/2025/04/27/society/kefal...,Κεφαλλονιά: Φωτιά σε δασική έκταση στη περιοχή...,27-04-2025,15:02,Κοινωνία,Newsroom,Για την κατάσβεση της φωτιάς στην Κεφαλλονιά ε...,Την κινητοποίηση της Πυροσβεστικής προκάλεσε φ...,https://www.tovima.gr/wp-content/uploads/2025/...,Για την κατάσβεση της φωτιάς στην Κεφαλλονιά ε...
2,https://www.tovima.gr/2025/03/13/society/fotia...,Φωτιά σε αγροτοδασική έκταση στα Χανιά – Μήνυμ...,13-03-2025,08:23,Κοινωνία,Newsroom,Για την κατάσβεση της φωτιά στα Χανιά επιχειρο...,Την κινητοποίηση της πυροσβεστικής προκάλεσε π...,https://www.tovima.gr/wp-content/uploads/2024/...,Για την κατάσβεση της φωτιά στα Χανιά επιχειρο...
3,https://www.tovima.gr/2024/09/28/society/ksant...,Ξάνθη: Φωτιά σε δασική έκταση – Επιχειρούν ενά...,28-09-2024,16:08,Κοινωνία,Newsroom,Μάχη με τις φλόγες της φωτιάς σε δασική έκταση...,"Φωτιά ξέσπασε σε δασική έκταση στην Ξάνθη, κιν...",https://www.tovima.gr/wp-content/uploads/2024/...,Μάχη με τις φλόγες της φωτιάς σε δασική έκταση...
4,https://www.tovima.gr/2024/09/01/society/fotia...,Φωτιά τώρα στη Φθιώτιδα – Καίγεται δασική έκταση,01-09-2024,18:12,Κοινωνία,Newsroom,Φωτιά τώρα ξέσπασε στη Φθιώτιδα - Καίγεται δασ...,"Φωτιά τώρα ξέσπασε στη Φθιώτιδα, στην περιοχή ...",https://www.tovima.gr/wp-content/uploads/2024/...,Φωτιά τώρα ξέσπασε στη Φθιώτιδα - Καίγεται δασ...
...,...,...,...,...,...,...,...,...,...,...
95,https://www.tovima.gr/2023/05/14/society/rodos...,Ρόδος: Πυρκαγιά κοντά σε δασική έκταση,14-05-2023,16:03,Κοινωνία,Newsroom,Επί ποδός η πυροσβεστική στη Ρόδο μετά από φωτ...,Μεγάλη κινητοποίηση των πυροσβεστικών δυνάμεων...,https://www.tovima.gr/wp-content/uploads/2023/...,Επί ποδός η πυροσβεστική στη Ρόδο μετά από φωτ...
96,https://www.tovima.gr/2023/04/27/society/pyrka...,Πυρκαγιά σε δασική έκταση στα Μέθανα,27-04-2023,15:20,Κοινωνία,Newsroom,Στο σημείο επιχειρούν 15 πυροσβέστες με 6 οχήμ...,Πυρκαγιά ξέσπασε σήμερα το μεσημέρι σε αγροτοδ...,https://www.tovima.gr/wp-content/uploads/2022/...,Στο σημείο επιχειρούν 15 πυροσβέστες με 6 οχήμ...
97,https://www.tovima.gr/2022/09/21/society/pyrka...,,,,,,,,,
98,https://www.tovima.gr/2022/08/06/society/lakon...,,,,,,,,,


In [ ]:
fire_articles_tovima_df = fire_articles_tovima_df.replace(r'^\s*$', pd.NA, regex=True)
fire_articles_tovima_df = fire_articles_tovima_df.dropna(subset=['title'])
fire_articles_tovima_df

,article_url,title,date,time,section,author,excert,text_body,feat_img_link,feat_img_caption
0,https://www.tovima.gr/2025/04/16/society/agrot...,Αγροτοδασική φωτιά στη Φθιώτιδα – Μεγάλη κινητ...,16-04-2025,14:51,Κοινωνία,Newsroom,Κοντά σε πευκόφυτη περιοχή στον Λόγγο του Αγίο...,Σε συναγερμό έχουν τεθεί οι πυροσβεστικές δυνά...,https://www.tovima.gr/wp-content/uploads/2024/...,Κοντά σε πευκόφυτη περιοχή στον Λόγγο του Αγίο...
1,https://www.tovima.gr/2025/04/27/society/kefal...,Κεφαλλονιά: Φωτιά σε δασική έκταση στη περιοχή...,27-04-2025,15:02,Κοινωνία,Newsroom,Για την κατάσβεση της φωτιάς στην Κεφαλλονιά ε...,Την κινητοποίηση της Πυροσβεστικής προκάλεσε φ...,https://www.tovima.gr/wp-content/uploads/2025/...,Για την κατάσβεση της φωτιάς στην Κεφαλλονιά ε...
2,https://www.tovima.gr/2025/03/13/society/fotia...,Φωτιά σε αγροτοδασική έκταση στα Χανιά – Μήνυμ...,13-03-2025,08:23,Κοινωνία,Newsroom,Για την κατάσβεση της φωτιά στα Χανιά επιχειρο...,Την κινητοποίηση της πυροσβεστικής προκάλεσε π...,https://www.tovima.gr/wp-content/uploads/2024/...,Για την κατάσβεση της φωτιά στα Χανιά επιχειρο...
3,https://www.tovima.gr/2024/09/28/society/ksant...,Ξάνθη: Φωτιά σε δασική έκταση – Επιχειρούν ενά...,28-09-2024,16:08,Κοινωνία,Newsroom,Μάχη με τις φλόγες της φωτιάς σε δασική έκταση...,"Φωτιά ξέσπασε σε δασική έκταση στην Ξάνθη, κιν...",https://www.tovima.gr/wp-content/uploads/2024/...,Μάχη με τις φλόγες της φωτιάς σε δασική έκταση...
4,https://www.tovima.gr/2024/09/01/society/fotia...,Φωτιά τώρα στη Φθιώτιδα – Καίγεται δασική έκταση,01-09-2024,18:12,Κοινωνία,Newsroom,Φωτιά τώρα ξέσπασε στη Φθιώτιδα - Καίγεται δασ...,"Φωτιά τώρα ξέσπασε στη Φθιώτιδα, στην περιοχή ...",https://www.tovima.gr/wp-content/uploads/2024/...,Φωτιά τώρα ξέσπασε στη Φθιώτιδα - Καίγεται δασ...
...,...,...,...,...,...,...,...,...,...,...
92,https://www.tovima.gr/2023/07/06/society/pyrka...,Πυρκαγιά σε δασική έκταση στο Μαρκόπουλο Ωρωπού,06-07-2023,14:32,Κοινωνία,Newsroom,Για την κατάσβεση της φωτιάς επιχειρούν μέχρι ...,Πυρκαγιά έχει ξεσπάσει σε δασική έκταση στο Μα...,https://www.tovima.gr/wp-content/uploads/2023/...,Για την κατάσβεση της φωτιάς επιχειρούν μέχρι ...
93,https://www.tovima.gr/2023/05/29/world/kanadas...,Καναδάς: Περίπου 18.000 άνθρωποι εγκατέλειψαν ...,29-05-2023,23:17,Κόσμος,Newsroom,Ο πρωθυπουργός του Καναδά Τζάστιν Τριντό υπογρ...,Χιλιάδες άνθρωποι υποχρεώθηκαν να εγκαταλείψου...,https://www.tovima.gr/wp-content/uploads/2023/...,Ο πρωθυπουργός του Καναδά Τζάστιν Τριντό υπογρ...
94,https://www.tovima.gr/2023/05/27/society/pyrka...,Πυρκαγιά σε αγροτοδασική έκταση στα Καλύβια Θο...,27-05-2023,18:19,Κοινωνία,Newsroom,Διακοπή της κυκλοφορίας στη λεωφόρο Αναβύσσου ...,Πυρκαγιά έχει ξεσπάσει σε αγροτοδασική έκταση ...,https://www.tovima.gr/wp-content/uploads/2023/...,Διακοπή της κυκλοφορίας στη λεωφόρο Αναβύσσου ...
95,https://www.tovima.gr/2023/05/14/society/rodos...,Ρόδος: Πυρκαγιά κοντά σε δασική έκταση,14-05-2023,16:03,Κοινωνία,Newsroom,Επί ποδός η πυροσβεστική στη Ρόδο μετά από φωτ...,Μεγάλη κινητοποίηση των πυροσβεστικών δυνάμεων...,https://www.tovima.gr/wp-content/uploads/2023/...,Επί ποδός η πυροσβεστική στη Ρόδο μετά από φωτ...


In [ ]:
fire_articles_tovima_df.to_csv('/content/drive/MyDrive/Final Essay ΕΔΔΕ.2 Αθηνιώτη Μιχαέλα/clean csv/forest_fires_tovima_articles_csv.csv', index=False)